In [1]:
#Importing all the required packages
import numpy as np
import os 
import cv2
import random
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D,Dense,BatchNormalization,Dropout
from tensorflow.keras.layers import Flatten
from keras.regularizers import l2
from keras.datasets import cifar10
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

C:\Users\Lenovo\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Load all the images from the dataset one by one in grayscale and extracting the 
# groud truth labels which are nothing but 7th character of each file name in hands directory
label=[]
images = []
for filename in os.listdir('hands'):
    img = cv2.imread(os.path.join('hands',filename),0).astype("float32")
#     img=np.array(img)
#     print(i)
#     i=i+1
#     a_min=np.min(img)
#     a_max=np.max(img)
#     img=255*(img-a_min)/(a_max-a_min)
    if img is not None:
        images.append(img)
        p=filename[6]
        if(ord(p)>=97 and ord(p)<=122):   #Labeling the images from 0 rough 36
            label.append(ord(p)-97+10)
        else:
            label.append(int(p))

In [ ]:
#convert list of images into numpy arrays
images=np.array(images)
label=np.array(label)

In [ ]:
images=images.reshape(len(images),670,600,1)
label=label.reshape(len(label),1)

In [ ]:
#Here we randomize the images along with their corresponding labels
combined = list(zip(images, label))
random.shuffle(combined)
images[:], label[:] = zip(*combined)

In [ ]:
#Splitting dataset into training and testing data
x_train=images[:2200]
x_test=images[2200:]
y_train=label[:2200]
y_test=label[2200:]

In [ ]:
# Model Definition
model = Sequential()
model.add(Conv2D(32, kernel_size = (5, 5), activation='relu',kernel_regularizer=l2(0.001), input_shape=(670, 600, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(5,5), kernel_regularizer=l2(0.001),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(5,5), kernel_regularizer=l2(0.001),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(5,5), kernel_regularizer=l2(0.001),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size=(5,5), kernel_regularizer=l2(0.001),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(36, activation = 'softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer="Adam", metrics=['accuracy'])

In [ ]:
# Training
model.fit(x=x_train, y=y_train, epochs=100,  validation_split=0.2)

In [ ]:
#Testing
model.evaluate(x_test,y_test)

In [ ]:
#Saving the weights of the model so that we can use the in predicting new images
model.save_weights("finalweights.h5")